In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Populate Media

In [9]:
get_links = pd.read_csv("https://docs.google.com/spreadsheets/d/1nE_v4AFkLM76TdWcAbU1HRWgQ-gGKYa9ZMBbm7MM0V0/export?gid=0&format=csv")
get_links.head()

,Anggota,Link,Kategori,Unnamed: 3,Unnamed: 4
0,Ghulam,https://industri.kontan.co.id/news/pemerintah-...,Finance,NaN,NaN
1,NaN,https://news.okezone.com/read/2023/10/29/337/2...,Incident,NaN,NaN
2,NaN,https://www.cnnindonesia.com/ekonomi/202306081...,Incident,NaN,NaN
3,NaN,https://www.antaranews.com/video/3366141/ini-d...,Incident,NaN,NaN
4,NaN,https://haji.kemenag.go.id/v5/detail/tingkatka...,Operational,NaN,NaN


In [10]:
get_links[get_links['Link'].str.startswith('https://www.cnbcindonesia.com/news/20190611084856-4-77493/cerita-lion-air-yang')]['Link']

75    https://www.cnbcindonesia.com/news/20190611084...
Name: Link, dtype: object

In [3]:
# Validate https:// prefix 
if len(get_links[~get_links["Link"].str.startswith("https://")]) > 0:
    raise Exception(f"Found link without https prefix : \n{get_links[~get_links["Link"].str.startswith("https://")]}")

In [4]:
# Populate
media = set(get_links.Link.apply(lambda x : x.replace("https://", "").split("/")[0]))
media

{'20.detik.com',
 'asumsi.co',
 'banyumas.tribunnews.com',
 'bnn.go.id',
 'daerah.sindonews.com',
 'diskominfo.bandaacehkota.go.id',
 'ekbis.sindonews.com',
 'ekonomi.bisnis.com',
 'era.id',
 'fajarpapua.com',
 'haji.kemenag.go.id',
 'hargo.co.id',
 'indoaviation.asia',
 'industri.kontan.co.id',
 'jatim.antaranews.com',
 'jubi.id',
 'kaltimkita.com',
 'kbr.id',
 'koranmakassar.com',
 'makassar.antaranews.com',
 'megapolitan.kompas.com',
 'money.kompas.com',
 'nasional.kontan.co.id',
 'news.detik.com',
 'news.okezone.com',
 'papuabarat.tribunnews.com',
 'popularitas.com',
 'portal.merauke.go.id',
 'regional.kompas.com',
 'regional.kontan.co.id',
 'sorongraya.inews.id',
 'sumatra.bisnis.com',
 'sumbar.kemenag.go.id',
 'sumsel.antaranews.com',
 'tangerang.tribunnews.com',
 'tirto.id',
 'travel.detik.com',
 'travel.kompas.com',
 'tribratanews.polri.go.id',
 'wahananews.co',
 'wartakota.tribunnews.com',
 'www.antaranews.com',
 'www.bbc.com',
 'www.bisnis.com',
 'www.bloombergtechnoz.com',
 

### Scrape Article Content

In [ ]:
soup = BeautifulSoup(response.content, 'lxml')

In [36]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.google.com/', # The 'Referer' header tells the server where you came from
    'DNT': '1', # Do Not Track
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}
data = []
link_len = len(get_links['Link'])
try:
    for i, link in enumerate(get_links['Link'][150:], start=1):
        response = requests.get(link, headers=headers)
        response.raise_for_status() 

        soup = BeautifulSoup(response.content, 'html.parser')
        title_element = soup.find('h1', class_='article-title')
        article_title = title_element.get_text(strip=True) if title_element else "Title not found"

        content_container = soup.find('div', id='article-body')

        article_text = ""
        if content_container:
            paragraphs = content_container.find_all('p')
            article_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        else:
            article_text = "Article content not found."
        data.append((article_title, article_text))
        
        print(f"({i}/{link_len}) {link} SUCCESS")
except requests.exceptions.RequestException as e:
    print(e)

(1/153) https://kbr.id/articles/indeks/lion-air-kroscek-dugaan-mesin-rusak SUCCESS
(2/153) https://www.inews.id/finance/bisnis/begini-kronologi-pesawat-lion-air-jt-507-yang-gagal-terbang SUCCESS
(3/153) https://regional.kontan.co.id/news/ada-bunyi-ledakan-lion-air-gagal-terbang SUCCESS


In [63]:
article = "https://ekbis.sindonews.com/read/1032941/34/insiden-penumpang-lion-air-lempar-handphone-berasap-ini-penjelasan-manajemen-1677409355"
requests.get(article)

<Response [403]>

In [64]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.google.com/', # The 'Referer' header tells the server where you came from
    'DNT': '1', # Do Not Track
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}
req = requests.get(article, headers=headers)
req.raise_for_status()
soup = BeautifulSoup(req.content, 'html.parser')
article_title = soup.find('title').text if soup.find('title') else "Title not found"
article_title

'Insiden Penumpang Lion Air Lempar Handphone Berasap, Ini Penjelasan Manajemen'

In [65]:
par = soup.find_all('p')
# for p in par:
#     print()
#     print(p.get_text())
# "\n\n".join(par) 
"\n".join(list(map(lambda x : x.get_text(), par)))

''

In [23]:
req.content

b'<!DOCTYPE html>\r\n<html lang="id-ID">\r\n\r\n<head>\r\n    \r\n\t<title>Terancam 8 Tahun Penjara, Ini Fakta-fakta Penumpang Lion Air Teriak Bom di Pesawat</title>\r\n\t<meta charset="utf-8">\r\n\t<meta name="robots" content="index,follow" />\r\n\t<meta name="googlebot" content="index,follow" />\r\n\t<meta name="googlebot-news" content="index,follow" />\t\r\n\t<meta name="title" content="Terancam 8 Tahun Penjara, Ini Fakta-fakta Penumpang Lion Air Teriak Bom di Pesawat">\r\n\t<meta name="google-site-verification" content="XXtXnqkl_OEcVprI_BjJzK2S-eUY9eh0gWxRmzrUP0w" />\r\n\t<meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=0">\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests" />\r\n\t<meta http-equiv="content-language" content="In-Id" />\r\n\t<meta http-equiv="cache-control" content="max-age=0" />\r\n\t<meta http-equiv="cache-con

In [12]:
soup = BeautifulSoup(req.content)

In [15]:
main_div = soup.find('div', class_="tmpt-desk-kon")

In [47]:
par = soup.find_all('p')
for p in par:
    print()
    print(p.get_text())


Reporter: Leni Wandira | Editor: Wahyu T.Rahmawati

KONTAN.CO.ID - JAKARTA. Lion Group mendukung penuh kebijakan Pemerintah melalui Kementerian Perhubungan yang menurunkan tarif tiket pesawat kelas ekonomi pada penerbangan domestik selama periode Natal dan Tahun Baru (Nataru) 2024/2025, yaitu mulai 19 Desember 2024 hingga 3 Januari 2025.  

Kebijakan ini sesuai dengan arahan Presiden Prabowo Subianto yang bertujuan untuk mengurangi beban biaya perjalanan udara bagi masyarakat sekaligus mendorong perekonomian Indonesia. 

Lion Group menyambut baik kebijakan penurunan tarif 10% ini sebagai bagian dari upaya memperkuat konektivitas nasional, mendukung mobilitas masyarakat, serta mempercepat pembangunan ekonomi negara melalui sektor transportasi udara.  

"Dengan diterapkannya penurunan tarif di seluruh bandar udara di Indonesia, Lion Group berharap kebijakan ini dapat memberikan manfaat yang nyata bagi masyarakat, terutama yang bepergian selama periode liburan Nataru," kata Corporate Com

In [16]:
if main_div:
    paragraphs = main_div.find_all('p')
    
    for p in paragraphs:
        print(p.get_text())

Reporter: Leni Wandira | Editor: Wahyu T.Rahmawati
KONTAN.CO.ID - JAKARTA. Lion Group mendukung penuh kebijakan Pemerintah melalui Kementerian Perhubungan yang menurunkan tarif tiket pesawat kelas ekonomi pada penerbangan domestik selama periode Natal dan Tahun Baru (Nataru) 2024/2025, yaitu mulai 19 Desember 2024 hingga 3 Januari 2025.  
Kebijakan ini sesuai dengan arahan Presiden Prabowo Subianto yang bertujuan untuk mengurangi beban biaya perjalanan udara bagi masyarakat sekaligus mendorong perekonomian Indonesia. 
Lion Group menyambut baik kebijakan penurunan tarif 10% ini sebagai bagian dari upaya memperkuat konektivitas nasional, mendukung mobilitas masyarakat, serta mempercepat pembangunan ekonomi negara melalui sektor transportasi udara.  
"Dengan diterapkannya penurunan tarif di seluruh bandar udara di Indonesia, Lion Group berharap kebijakan ini dapat memberikan manfaat yang nyata bagi masyarakat, terutama yang bepergian selama periode liburan Nataru," kata Corporate Communic